<a href="https://colab.research.google.com/github/mostafa-ja/mal_adv3/blob/main/3_adverserial_attacks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from scipy import sparse
import gdown
import numpy as np
import matplotlib.pyplot as plt
import pickle
import torch.nn.functional as F
import random


In [2]:
download_link = 'https://drive.google.com/uc?id=12iud4h19CZst4exbr3U2A9iDxBYvZ5U_'
output_filepath = '/content/'
gdown.download(download_link, output_filepath)

Downloading...
From: https://drive.google.com/uc?id=12iud4h19CZst4exbr3U2A9iDxBYvZ5U_
To: /content/X_redefined_sparse_matrix.npz
100%|██████████| 2.31M/2.31M [00:00<00:00, 53.4MB/s]


'/content/X_redefined_sparse_matrix.npz'

In [3]:
download_link = 'https://drive.google.com/uc?id=1IhrcT3jHqlPrw2KvQ5vJkBgozxcJ1cJm'
output_filepath = '/content/'
gdown.download(download_link, output_filepath)

Downloading...
From: https://drive.google.com/uc?id=1IhrcT3jHqlPrw2KvQ5vJkBgozxcJ1cJm
To: /content/labels.pt
100%|██████████| 517k/517k [00:00<00:00, 24.7MB/s]


'/content/labels.pt'

In [4]:
download_link = 'https://drive.google.com/uc?id=13o5n06UpMDOhtk4u7B_RBSWa3kiiGXFs'
output_filepath = '/content/'
gdown.download(download_link, output_filepath)

Downloading...
From: https://drive.google.com/uc?id=13o5n06UpMDOhtk4u7B_RBSWa3kiiGXFs
To: /content/DNN_params.pth
100%|██████████| 8.17M/8.17M [00:00<00:00, 112MB/s]


'/content/DNN_params.pth'

In [5]:
download_link = 'https://drive.google.com/uc?id=1PxFOLBnQAlX-EOsqkhGCSd1T3ykAD0-4'
output_filepath = '/content/'
gdown.download(download_link, output_filepath)

Downloading...
From: https://drive.google.com/uc?id=1PxFOLBnQAlX-EOsqkhGCSd1T3ykAD0-4
To: /content/vocab.pkl
100%|██████████| 9.18M/9.18M [00:00<00:00, 85.2MB/s]


'/content/vocab.pkl'

In [6]:
# Load the dictionary from the file
with open('vocab.pkl', 'rb') as f:
    vocab = pickle.load(f)

for i, (key, value) in enumerate(vocab.items()):
    print((key, value))
    if i >= 5:
        break

('android/media/mediaplayer->start', 141045)
('android/app/activity->setcontentview', 140900)
('android/os/vibrator->cancel', 141093)
('android.permission.vibrate', 140720)
('android.hardware.touchscreen', 137091)
('android.intent.action.main', 138335)


In [7]:
# Load dataset
X_redefined = sparse.load_npz("X_redefined_sparse_matrix.npz")
labels_tensor = torch.load('labels.pt')

In [8]:
# Split data into train, validation, and test sets with stratified sampling
X_train_val, X_test, labels_train_val, labels_test = train_test_split(X_redefined, labels_tensor, test_size=0.2, stratify=labels_tensor, random_state=42)
X_train, X_val, labels_train, labels_val = train_test_split(X_train_val, labels_train_val, test_size=0.2, stratify=labels_train_val, random_state=42)

# Combine features and labels into datasets
train_dataset = TensorDataset(torch.tensor(X_train.toarray(), dtype=torch.float32), labels_train)
val_dataset = TensorDataset(torch.tensor(X_val.toarray(), dtype=torch.float32), labels_val)
test_dataset = TensorDataset(torch.tensor(X_test.toarray(), dtype=torch.float32), labels_test)


In [9]:
# Define the DataLoader for training, validation, and test sets
batch_size = 256
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [10]:
import torch
import torch.nn as nn

class MalwareDetectionModel(nn.Module):
    def __init__(self, input_size=10000, hidden_1_size=200, hidden_2_size=200, num_labels=2, dropout_prob=0.6):
        super(MalwareDetectionModel, self).__init__()

        self.input_size = input_size
        self.hidden_1_size = hidden_1_size
        self.hidden_2_size = hidden_2_size
        self.num_labels = num_labels
        self.dropout_prob = dropout_prob

        self.fc1 = nn.Linear(input_size, hidden_1_size)
        self.relu1 = nn.ReLU()
        self.dropout1 = nn.Dropout(dropout_prob)
        self.fc2 = nn.Linear(hidden_1_size, hidden_2_size)
        self.relu2 = nn.ReLU()
        self.dropout2 = nn.Dropout(dropout_prob)
        self.fc3 = nn.Linear(hidden_2_size, num_labels)
        self.log_softmax = nn.LogSoftmax(dim=1)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        x = self.log_softmax(x)
        return x


In [11]:
# Create an instance of your model
model = MalwareDetectionModel()

# Load model parameters
model.load_state_dict(torch.load('DNN_params.pth'))

<All keys matched successfully>

In [12]:
X,y = next(iter(test_loader))
print(X.shape)
print(y.shape)

torch.Size([256, 10000])
torch.Size([256, 1])




```
delta = torch.zeros_like(X, requires_grad=True)
for t in range(25):
    loss = nn.CrossEntropyLoss()(model(X + delta), y.view(-1).long())
    loss.backward()
    gradients = delta.grad.detach().sign() * (X < 0.5)
    delta.data = (delta + 0.02*delta.grad.detach().sign()).clamp(0.,1.)
    print(delta.data[0])
    delta.grad.zero_()
```



In [13]:
def round_x(x, round_threshold=0.5):
    """
    Rounds x by thresholding it according to round_threshold.
    :param x: input tensor
    :param round_threshold: threshold parameter
    :return: a tensor of 0s and 1s
    """
    return (x >= round_threshold).float()

def get_x0(x, initial_rounding_threshold=0.5, is_sample=False):
    """
    Helper function to randomly initialize the inner maximizer algorithm.
    Randomizes the input tensor while preserving its functionality.
    :param x: input tensor
    :param rounding_threshold: threshold for rounding
    :param is_sample: flag to sample randomly from feasible area
    :return: randomly sampled feasible version of x
    """
    if is_sample:
        rand_x = round_x(torch.rand(x.size()), initial_rounding_threshold=initial_rounding_threshold)
        return (rand_x.byte() | x.byte()).float()
    else:
        return x

def or_float_tensors(x_1, x_2):
    """
    ORs two float tensors by converting them to byte and back.
    :param x_1: tensor one
    :param x_2: tensor two
    :return: float tensor of 0s and 1s
    """
    return (x_1.byte() | x_2.byte()).float()


def xor_float_tensors(x_1, x_2):
    """
    XORs two float tensors by converting them to byte and back
    Note that byte() takes the first 8 bit after the decimal point of the float
    e.g., 0.0 ==> 0
          0.1 ==> 0
          1.1 ==> 1
        255.1 ==> 255
        256.1 ==> 0
    Subsequently the purpose of this function is to map 1s float tensors to 1
    and those of 0s to 0. I.e., it is meant to be used on tensors of 0s and 1s.

    :param x_1: tensor one
    :param x_2: tensor two
    :return: float tensor of 0s and 1s.
    """
    return (x_1.byte() ^ x_2.byte()).float()

def get_loss(x,y,model):
    criterion = nn.CrossEntropyLoss(reduction='none')
    outputs = model(x)
    loss = criterion(outputs, y.view(-1).long())
    _, predicted = torch.topk(outputs, k=1)
    done = (predicted != y).squeeze()

    return loss, done





```
x_1 = torch.tensor([1.0, 1.0,  1.0, 1.0, 1.0, 0.0,  0.0, 0.2])
x_2 = torch.tensor([0.0, 0.6, -0.2, 0.9, 1.9, 1.0, -0.2, 0.3])
result = or_float_tensors(x_1, x_2)
             tensor([1., 1., 1., 1., 1., 1., 0., 0.])

```



In [28]:
def dfgsm_k(x, y, model, k=25, epsilon=0.02, alpha=1., initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    FGSM^k with deterministic rounding
    :param y: ground truth labels
    :param x: feature vector
    :param model: neural network model
    :param k: number of steps
    :param epsilon: update value in each direction
    :param alpha: hyperparameter for controlling the portionate of rounding
    :param initial_rounding_threshold: threshold parameter for rounding the initial x_next
    :param round_threshold: threshold parameter for rounding
    :param is_report_loss_diff: flag to report loss difference
    :param is_sample: flag to sample randomly from the feasible area
    :return: the adversarial version of x according to dfgsm_k (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        # Find the next sample
        x_next = x_next + epsilon * torch.sign(grad_vars[0].data)

        # Projection
        x_next = torch.clamp(x_next, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size()) * alpha
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        #print(f"Natural loss: {loss_natural.mean():.4f}, Adversarial loss: {loss_adv.mean():.4f}, Difference: {(loss_adv.mean() - loss_natural.mean()):.4f}")
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"rFGSM: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next


In [ ]:
x_adv = dfgsm_k(X,y,model)

rFGSM: attack effectiveness 100.000%.


In [ ]:
x_adv = dfgsm_k(X,y,model)

Natural loss: 0.0060, Adversarial loss: 2221.2881, Difference: 2221.2820
rFGSM: attack effectiveness 100.000%.


In [ ]:
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

tensor(26.6289)
tensor(3986.6562)


In [ ]:
x_adv = dfgsm_k(X,y,model,random=True)

Natural loss: 0.0060, Adversarial loss: 1290.9233, Difference: 1290.9174
rFGSM: attack effectiveness 100.000%.


In [ ]:
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

tensor(26.6289)
tensor(2741.3125)


In [ ]:
x_adv = dfgsm_k(X,y,model,k=1,random=True)


Natural loss: 0.0060, Adversarial loss: 38.6829, Difference: 38.6769
rFGSM: attack effectiveness 100.000%.


In [ ]:
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

tensor(26.6289)
tensor(120.8047)


In [ ]:
for alpha in range(1, 52, 10):
  print('alpha = ',alpha)
  x_adv = dfgsm_k(X, y, model, k=20, epsilon=0.02, alpha=alpha,random=True)
  print(X.sum()/batch_size)
  print(x_adv.sum()/batch_size)
  print('***************************')

alpha =  1
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(2189.4138)
***************************
alpha =  11
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(222.7241)
***************************
alpha =  21
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(132.2562)
***************************
alpha =  31
rFGSM: attack effectiveness 99.507%.
tensor(28.0148)
tensor(98.3202)
***************************
alpha =  41
rFGSM: attack effectiveness 94.581%.
tensor(28.0148)
tensor(80.3054)
***************************
alpha =  51
rFGSM: attack effectiveness 91.626%.
tensor(28.0148)
tensor(70.1970)
***************************


In [ ]:
total_samples = 0
total_attack_success = 0
total_features_x = 0
total_features_x_adv = 0

for X, y in test_loader:
    batch_size = X.size(0)
    x_adv = dfgsm_k(X, y, model, k=20, epsilon=0.0005, random=True,is_report_loss_diff=False)

    # Compute mean number of features
    total_features_x += X.sum().item()
    total_features_x_adv += x_adv.sum().item()

    # Compute attack effectiveness for this batch
    outputs_adv = model(x_adv)
    _, predicted_adv = torch.topk(outputs_adv, k=1)
    total_attack_success += torch.sum(predicted_adv != y).item()
    total_samples += batch_size

# Compute mean number of features
mean_features_x = total_features_x / total_samples
mean_features_x_adv = total_features_x_adv / total_samples

# Compute mean attack effectiveness
mean_attack_effectiveness = (total_attack_success / total_samples) * 100

# Print results
print('Mean number of features (X):', mean_features_x)
print('Mean number of features (X_adv):', mean_features_x_adv)
print('Mean attack effectiveness:', mean_attack_effectiveness)


Mean number of features (X): 27.5364492500872
Mean number of features (X_adv): 76.25450529008255
Mean attack effectiveness: 98.64356857729722


In [ ]:
total_samples = 0
total_attack_success = 0
total_features_x = 0
total_features_x_adv = 0

for X, y in test_loader:
    batch_size = X.size(0)
    x_adv = dfgsm_k(X, y, model, k=20, epsilon=0.02, alpha=40,random=True,is_report_loss_diff=False)

    # Compute mean number of features
    total_features_x += X.sum().item()
    total_features_x_adv += x_adv.sum().item()

    # Compute attack effectiveness for this batch
    outputs_adv = model(x_adv)
    _, predicted_adv = torch.topk(outputs_adv, k=1)
    total_attack_success += torch.sum(predicted_adv != y).item()
    total_samples += batch_size

# Compute mean number of features
mean_features_x = total_features_x / total_samples
mean_features_x_adv = total_features_x_adv / total_samples

# Compute mean attack effectiveness
mean_attack_effectiveness = (total_attack_success / total_samples) * 100

# Print results
print('Mean number of features (X):', mean_features_x)
print('Mean number of features (X_adv):', mean_features_x_adv)
print('Mean attack effectiveness:', mean_attack_effectiveness)

Mean number of features (X): 27.5364492500872
Mean number of features (X_adv): 81.65845056776344
Mean attack effectiveness: 96.50428244777738


In [ ]:
def dfgsm_k2(x, y, model, k=25, epsilon=0.02, alpha=1., initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    FGSM^k with deterministic rounding
    :param y: ground truth labels
    :param x: feature vector
    :param model: neural network model
    :param k: number of steps
    :param epsilon: update value in each direction
    :param alpha: hyperparameter for controlling the portionate of rounding
    :param initial_rounding_threshold: threshold parameter for rounding the initial x_next
    :param round_threshold: threshold parameter for rounding
    :param is_report_loss_diff: flag to report loss difference
    :param is_sample: flag to sample randomly from the feasible area
    :return: the adversarial version of x according to dfgsm_k (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        #print(torch.sign(grad_vars[0].data).clamp(min=0))
        # Find the next sample
        x_next = x_next + epsilon * (torch.sign(grad_vars[0].data).clamp(min=0))

        # Projection
        x_next = torch.clamp(x_next, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size()) * alpha
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        #print(f"Natural loss: {loss_natural.mean():.4f}, Adversarial loss: {loss_adv.mean():.4f}, Difference: {(loss_adv.mean() - loss_natural.mean()):.4f}")
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"rFGSM: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next

In [ ]:
x_adv = dfgsm_k2(X[:1],y[:1],model)

rFGSM: attack effectiveness 100.000%.


In [ ]:
for alpha in range(1, 52, 10):
  print('alpha = ',alpha)
  x_adv = dfgsm_k2(X, y, model, k=20, epsilon=0.02, alpha=alpha,random=True)
  print(X.sum()/batch_size)
  print(x_adv.sum()/batch_size)
  print('***************************')

alpha =  1
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(2246.3103)
***************************
alpha =  11
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(228.2020)
***************************
alpha =  21
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(132.3547)
***************************
alpha =  31
rFGSM: attack effectiveness 99.507%.
tensor(28.0148)
tensor(98.7537)
***************************
alpha =  41
rFGSM: attack effectiveness 98.030%.
tensor(28.0148)
tensor(82.3054)
***************************
alpha =  51
rFGSM: attack effectiveness 88.177%.
tensor(28.0148)
tensor(71.4335)
***************************


In [ ]:
total_samples = 0
total_attack_success = 0
total_features_x = 0
total_features_x_adv = 0

for X, y in test_loader:
    batch_size = X.size(0)
    x_adv = dfgsm_k2(X, y, model, k=20, epsilon=0.02, alpha=40,random=True,is_report_loss_diff=False)

    # Compute mean number of features
    total_features_x += X.sum().item()
    total_features_x_adv += x_adv.sum().item()

    # Compute attack effectiveness for this batch
    outputs_adv = model(x_adv)
    _, predicted_adv = torch.topk(outputs_adv, k=1)
    total_attack_success += torch.sum(predicted_adv != y).item()
    total_samples += batch_size

# Compute mean number of features
mean_features_x = total_features_x / total_samples
mean_features_x_adv = total_features_x_adv / total_samples

# Compute mean attack effectiveness
mean_attack_effectiveness = (total_attack_success / total_samples) * 100

# Print results
print('Mean number of features (X):', mean_features_x)
print('Mean number of features (X_adv):', mean_features_x_adv)
print('Mean attack effectiveness:', mean_attack_effectiveness)

Mean number of features (X): 27.5364492500872
Mean number of features (X_adv): 82.96155485796226
Mean attack effectiveness: 96.48878037437507


.clone(): This method creates a deep copy of the tensor, including its data and gradients (if any). It essentially creates a new tensor with the same data and properties as the original tensor. If the original tensor is part of a computation graph and requires gradients, the cloned tensor will also be part of the same computation graph and will require gradients. Any changes made to the cloned tensor will not affect the original tensor, and vice versa.

.copy(): This method creates a shallow copy of the tensor. It only copies the data, not the computational graph or gradients. Therefore, the copied tensor will be detached from any computation graph and will not require gradients, even if the original tensor did. Changes made to the copied tensor will not affect the original tensor, but changes in the original tensor's data will be reflected in the copied tensor.



In [ ]:
def bga_k(x, y, model, k=25, alpha=1., is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit gradient ascent
    :param x: feature vector
    :param y: ground truth labels
    :param model: neural network model
    :param k: number of steps
    :param alpha: hyperparameter for controlling updates
    :param is_report_loss_diff: flag to report loss difference
    :param use_sample: flag to sample randomly from the feasible area
    :return: the adversarial version of x according to bga_k (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize worst loss and corresponding adversarial samples
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # Book-keeping
    sqrt_m = torch.sqrt(torch.tensor([x.size()[1]], dtype=torch.float))

    # Multi-step with gradients
    for t in range(k):
        if t == 0:
            # Initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # Compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # Compute the updates
            # torch.norm(grad_data, 2, 1), 2:the L2-norm , 1:the norm along dimension 1
            x_update = (sqrt_m * (1. - 2. * x_next) * grad_data >= (alpha * torch.norm(grad_data, 2, 1).unsqueeze(1))).float()

            # Find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print(f"Natural loss: {loss_natural.mean():.4f}, Adversarial loss: {loss_worst.mean():.4f}, Difference: {(loss_worst.mean() - loss_natural.mean()):.4f}")
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"bga_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    return x_worst


In [ ]:
x_adv = bga_k(X, y, model, k=25, is_report_loss_diff=True, use_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

bga_k: attack effectiveness 100.000%.
tensor(26.6289)
tensor(2297.2891)


In [ ]:
for alpha in range(1,6):
  print('alpha = ',alpha)
  x_adv = bga_k(X, y, model, k=25, alpha=alpha, is_report_loss_diff=True, use_sample=False)
  print(X.sum()/batch_size)
  print(x_adv.sum()/batch_size)
  print('***************************')

alpha =  1
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(2289.4089)
***************************
alpha =  2
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(577.0690)
***************************
alpha =  3
rFGSM: attack effectiveness 100.000%.
tensor(28.0148)
tensor(185.2315)
***************************
alpha =  4
rFGSM: attack effectiveness 95.567%.
tensor(28.0148)
tensor(71.5714)
***************************
alpha =  5
rFGSM: attack effectiveness 1.478%.
tensor(28.0148)
tensor(28.0887)
***************************


In [65]:
def bca_k(x, y, model, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent
    :param use_sample:
    :param is_report_loss_diff:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param k: num of steps
    :return: the adversarial version of x according to bca_k (tensor)
    """

    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    loss = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates (can be made more efficient than this)
            aug_grad = (1. - 2. * x_next) * grad_data
            val, _ = torch.topk(aug_grad, 1)
            x_update = (aug_grad >= val.expand_as(aug_grad)).float()

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %(loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"bca_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")



    return x_worst

In [ ]:
x_adv = bca_k(X, y, model, k=3, is_report_loss_diff=True, use_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

bca_k: attack effectiveness 44.922%.
tensor(26.6289)
tensor(28.5430)


In [ ]:
for k in range(5, 55, 5):
  print('k = ',k)
  x_adv = bca_k(X, y, model, k=k, is_report_loss_diff=True, use_sample=False)
  print(X.sum()/batch_size)
  print(x_adv.sum()/batch_size)
  print('***************************')

k =  5
bca_k: attack effectiveness 91.406%.
tensor(26.6289)
tensor(30.5039)
***************************
k =  10
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(35.3906)
***************************
k =  15
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(40.2031)
***************************
k =  20
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(45.0078)
***************************
k =  25
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(49.8125)
***************************
k =  30
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(54.6172)
***************************
k =  35
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(59.4219)
***************************
k =  40
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(64.2266)
***************************
k =  45
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(69.0312)
***************************
k =  50
bca_k: attack effectiveness 97.266%.
tensor(26.6

In [ ]:
a = torch.rand((10,10000))
a

tensor([[0.2620, 0.9656, 0.6516,  ..., 0.0038, 0.3084, 0.7131],
        [0.5588, 0.2493, 0.0950,  ..., 0.9978, 0.9718, 0.1974],
        [0.7188, 0.9575, 0.2771,  ..., 0.1128, 0.0172, 0.7418],
        ...,
        [0.0582, 0.2115, 0.6837,  ..., 0.8870, 0.7203, 0.6274],
        [0.7887, 0.0591, 0.3733,  ..., 0.8148, 0.6377, 0.2545],
        [0.6163, 0.1027, 0.4877,  ..., 0.8024, 0.8279, 0.1123]])

In [ ]:
val, index = torch.topk(a, 10)

In [ ]:
val[:,-1:]

tensor([[0.9994],
        [0.9992],
        [0.9989],
        [0.9990],
        [0.9992],
        [0.9993],
        [0.9993],
        [0.9988],
        [0.9993],
        [0.9990]])

In [ ]:
def bca_k2(x, y, model, k=25, alpha=1, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent
    :param use_sample:
    :param is_report_loss_diff:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param k: num of steps
    :param alpha: top k gradients
    :return: the adversarial version of x according to bca_k (tensor)
    """

    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    loss = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            grad_vars = torch.autograd.grad(loss.mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates (can be made more efficient than this)
            aug_grad = (1. - 2. * x_next) * grad_data
            val, _ = torch.topk(aug_grad, alpha)
            x_update = (aug_grad >= val[:,-1:].expand_as(aug_grad)).float()

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %(loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"bca_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")



    return x_worst

In [ ]:
for k in range(2, 9):
  print('k = ',k)
  x_adv = bca_k2(X, y, model, k=k, alpha=5, is_report_loss_diff=True, use_sample=False)
  print(X.sum()/batch_size)
  print(x_adv.sum()/batch_size)
  print('***************************')

k =  2
bca_k: attack effectiveness 94.922%.
tensor(26.6289)
tensor(31.5430)
***************************
k =  3
bca_k: attack effectiveness 99.609%.
tensor(26.6289)
tensor(36.5195)
***************************
k =  4
bca_k: attack effectiveness 99.609%.
tensor(26.6289)
tensor(41.4805)
***************************
k =  5
bca_k: attack effectiveness 99.609%.
tensor(26.6289)
tensor(46.4453)
***************************
k =  6
bca_k: attack effectiveness 99.609%.
tensor(26.6289)
tensor(51.4102)
***************************
k =  7
bca_k: attack effectiveness 99.609%.
tensor(26.6289)
tensor(56.3750)
***************************
k =  8
bca_k: attack effectiveness 100.000%.
tensor(26.6289)
tensor(61.3477)
***************************


In [ ]:
for i in range(1, 12):
  print('top k = ',i)
  x_adv = bca_k2(X, y, model, k=2, alpha=i, is_report_loss_diff=True, use_sample=False)
  print(X.sum()/batch_size)
  print(x_adv.sum()/batch_size)
  print('***************************')

top k =  1
bca_k: attack effectiveness 4.688%.
tensor(26.6289)
tensor(27.5664)
***************************
top k =  2
bca_k: attack effectiveness 37.500%.
tensor(26.6289)
tensor(28.5508)
***************************
top k =  3
bca_k: attack effectiveness 75.391%.
tensor(26.6289)
tensor(29.5352)
***************************
top k =  4
bca_k: attack effectiveness 91.016%.
tensor(26.6289)
tensor(30.5430)
***************************
top k =  5
bca_k: attack effectiveness 94.922%.
tensor(26.6289)
tensor(31.5430)
***************************
top k =  6
bca_k: attack effectiveness 96.094%.
tensor(26.6289)
tensor(32.5391)
***************************
top k =  7
bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(33.5234)
***************************
top k =  8
bca_k: attack effectiveness 98.828%.
tensor(26.6289)
tensor(34.5195)
***************************
top k =  9
bca_k: attack effectiveness 99.219%.
tensor(26.6289)
tensor(35.5078)
***************************
top k =  10
bca_k: attack eff

In [ ]:
def grosse_k(x, y, model, k=25, is_report_loss_diff=True, use_sample=False):
    """
    Multi-step bit coordinate ascent
    :param use_sample:
    :param is_report_loss_diff:
    :param y:
    :param x: (tensor) feature vector
    :param model: nn model
    :param k: num of steps
    :return: the adversarial version of x according to bca_k (tensor)
    """

    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # keeping worst loss
    loss_worst = loss_natural.clone()
    x_worst = x.clone()

    # multi-step with gradients
    output = None
    x_var = None
    x_next = None
    for t in range(k):
        if t == 0:
            # initialize starting point
            x_next = get_x0(x, use_sample)
        else:
            # compute gradient
            # ouput.shape=([batch_size, 2]) because of 2 neoruns, so we just use the output of the first neorun(benign)
            grad_vars = torch.autograd.grad(output[:, 0].mean(), x_var)
            grad_data = grad_vars[0].data

            # compute the updates (can be made more efficient than this)
            aug_grad = (1. - x_next) * grad_data
            val, _ = torch.topk(aug_grad, 1)
            x_update = (aug_grad >= val.expand_as(aug_grad)).float()

            # find the next sample with projection to the feasible set
            x_next = xor_float_tensors(x_update, x_next)
            x_next = or_float_tensors(x_next, x)

        # forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        output = model(x_var)
        loss = criterion(output, y.view(-1).long())

        # update worst loss and adversarial samples
        replace_flag = (loss.data > loss_worst)
        loss_worst[replace_flag] = loss.data[replace_flag]
        x_worst[replace_flag] = x_next[replace_flag]

    if is_report_loss_diff:
        #print("Natural loss (%.4f) vs Adversarial loss (%.4f), Difference: (%.4f)" %(loss_natural.mean(), loss_worst.mean(), loss_worst.mean() - loss_natural.mean()))
        outputs = model(x_worst)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"grosse_k: attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")



    return x_worst

In [ ]:
grosse_k(X, y, model, k=25, is_report_loss_diff=True, use_sample=False)

grosse_k: attack effectiveness 5.078%.


tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [1., 1., 1.,  ..., 1., 1., 1.]])

In [14]:
def pgd(x, y, model, k=25, step_length=0.02, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False):
    """
    Projected Gradient Descent (PGD) adversarial attack.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :param initial_rounding_threshold: Threshold parameter for rounding the initial x_next
    :param round_threshold: Threshold parameter for rounding
    :param random: Flag to generate random thresholds
    :param is_report_loss_diff: Flag to report loss difference
    :param is_sample: Flag to sample randomly from the feasible area
    :return: The adversarial version of x (tensor)
    """
    model.eval()

    # Compute natural loss
    criterion = nn.CrossEntropyLoss(reduction='none')
    loss_natural = criterion(model(x), y.view(-1).long())

    # Initialize starting point
    x_next = x.clone()
    x_next = get_x0(x_next, initial_rounding_threshold, is_sample)

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        grad_data = grad_vars[0].data
        gradients = grad_data * (x < 0.5)

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients)
        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm)
            perturbation[torch.isnan(perturbation)] = 0.
            perturbation[torch.isinf(perturbation)] = 1.
        elif norm == 'l1':
            #ignore the gradient of indice which is updated
            gradients = gradients * (x_next < 0.5)
            val, _ = torch.topk(gradients, 1)
            perturbation = torch.sign(gradients >= val.expand_as(gradients))
            # stop perturbing the examples that are successful to evade the victim
            outputs = model(x_next)
            _, predicted = torch.topk(outputs, k=1)
            done = (predicted != y).squeeze()
            if torch.all(done):
                break
            perturbation[done] = 0.
        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    # Rounding step
    if random:
       round_threshold = torch.rand(x_next.size())
    x_next = round_x(x_next, round_threshold=round_threshold)

    # Feasible projection
    x_next = or_float_tensors(x_next, x)

    # Compute adversarial loss
    loss_adv = criterion(model(x_next), y.view(-1).long()).data

    if is_report_loss_diff:
        outputs = model(x_next)
        _, predicted = torch.topk(outputs, k=1)
        done = (predicted != y).squeeze()
        print(f"PGD {norm}: Attack effectiveness {done.sum().item() / x.size()[0] * 100:.3f}%.")

    # Replace with natural if adversarial loss is higher
    replace_flag = (loss_adv < loss_natural).squeeze()
    x_next[replace_flag] = x[replace_flag]

    return x_next


In [38]:
x_adv = pgd(X, y, model, k=250, step_length=0.002, norm='linf', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

PGD: Attack effectiveness 100.000%.
tensor(26.6289)
tensor(3903.8359)


In [40]:
x_adv = pgd(X, y, model, k=200, step_length=0.05, norm='l2', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

PGD l2: Attack effectiveness 93.359%.
tensor(26.6289)
tensor(35.3633)


In [45]:
x_adv = pgd(X, y, model, k=227, step_length=0.05, norm='l2', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

PGD: Attack effectiveness 98.438%.
tensor(26.6289)
tensor(44.4180)


In [16]:
x_adv = pgd(X, y, model, k=50, step_length=1., norm='l1', initial_rounding_threshold=0.5, round_threshold=0.5, random=False, is_report_loss_diff=True, is_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

PGD: Attack effectiveness 100.000%.
tensor(26.6289)
tensor(29.4453)


In [72]:
x_adv = bca_k(X, y, model, k=53, is_report_loss_diff=True, use_sample=False)
print(X.sum()/batch_size)
print(x_adv.sum()/batch_size)

bca_k: attack effectiveness 97.266%.
tensor(26.6289)
tensor(76.7188)


In [41]:
x = X
steps_max=5
attack_list = ['linf', 'l2', 'l1']
is_sample = False
varepsilon = 1e-20

model.eval()
with torch.no_grad():
    loss, done = get_loss(x,y,model) #shape:[samples],[samples]
pre_loss = loss
n, red_n = x.shape[0], x.shape[1]
adv_x = x.detach().clone()
stop_flag = torch.zeros(n, dtype=torch.bool) #[samples]

for t in range(steps_max):
  num_sample_red = n - torch.sum(stop_flag)
  print('num_sample_red : ',num_sample_red)
  if num_sample_red <= 0:
      break

  red_label = y[~stop_flag]
  pertbx = []


  for norm in attack_list:
      if norm=='l1':
        l1 = pgd(adv_x[~stop_flag], red_label, model, k=50, step_length=1., norm='l1', is_sample=is_sample)
        pertbx.append(l1)
      elif norm=='l2':
        l2 = pgd(adv_x[~stop_flag], red_label, model, k=200, step_length=0.05, norm='l2', is_sample=is_sample)
        pertbx.append(l2)
      elif norm=='linf':
        linf = pgd(adv_x[~stop_flag], red_label, model, k=500, step_length=0.002, norm='linf', is_sample=is_sample)
        pertbx.append(linf)
      else :
        raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

  # here pertbx.shape = a list of (number of attacks  ,(num_sample_red,features))
  pertbx = torch.vstack(pertbx)
  # here pertbx.shape = a tensor (num_sample_red*number of attacks samples, features)

  with torch.no_grad():
    red_label_ext = torch.cat([red_label] * len(attack_list)) #(labels*number of attacks )
    loss, done = get_loss(pertbx, red_label_ext,model) #(labels*number of attacks )
    loss = loss.reshape(len(attack_list), num_sample_red).permute(1, 0) #(num_sample_red,number of attacks)
    done = done.reshape(len(attack_list), num_sample_red).permute(1, 0) #(num_sample_red,number of attacks)

    success_flag = torch.any(done, dim=-1) #(num_sample_red)
    # for a sample, if there is at least one successful attack, we will select the one with maximum loss;
    # while if no attacks evade the victim successful, all perturbed examples are reminded for selection

    done[~torch.any(done, dim=-1)] = 1 #loss.shape=done.shape=(samples,number of attacks)
    loss = (loss * done.to(torch.float)) + torch.min(loss) * (~done).to(torch.float) #(num_sample_red,number of attacks)
    pertbx = pertbx.reshape(len(attack_list), num_sample_red, red_n).permute([1, 0, 2])#(num_sample_red,attacks,features)
    _, indices = loss.max(dim=-1) # ans:(samples), max loss among attacks which worked, and max loss among all attacks for sample , none of them worked
    adv_x[~stop_flag] = pertbx[torch.arange(num_sample_red), indices]
    a_loss = loss[torch.arange(num_sample_red), indices]
    pre_stop_flag = stop_flag.clone()
    stop_flag[~stop_flag] = (torch.abs(pre_loss[~stop_flag] - a_loss) < varepsilon) | success_flag
    pre_loss[~pre_stop_flag] = a_loss

num_sample_red :  tensor(256)
PGD linf: Attack effectiveness 100.000%.
PGD l2: Attack effectiveness 93.359%.
PGD l1: Attack effectiveness 100.000%.
num_sample_red :  tensor(0)


In [43]:
print(X.sum()/batch_size)
print(adv_x.sum()/batch_size)

tensor(26.6289)
tensor(5542.4414)


In [59]:
def PGD_Max(x,y, model, attack_list = ['linf', 'l2', 'l1'],steps_max=5, is_sample = False, varepsilon = 1e-20):
    """
    PGD_Max adversarial attack.

    Args:
        x: Input data tensor (shape: [samples, features])
        y: Ground truth labels tensor (shape: [samples])
        model: Neural network model
        attack_list: List of norms for attacks (default: ['linf', 'l2', 'l1'])
        steps_max: Maximum number of steps (default: 5)
        is_sample: Flag to sample randomly from the feasible area (default: False)
        vaρεpsilon: Tolerance for stopping condition (default: 1e-20)

    Returns:
        Adversarial version of input data (tensor)
    """

    norm_params = {
        'l1': {'k': 50, 'step_length': 1.0},
        'l2': {'k': 200, 'step_length': 0.05},
        'linf': {'k': 500, 'step_length': 0.002}
    }

    model.eval()
    with torch.no_grad():
        loss, done = get_loss(x,y,model) #shape:[samples],[samples]

    pre_loss = loss
    n = x.shape[0]
    adv_x = x.detach().clone()
    stop_flag = torch.zeros(n, dtype=torch.bool) #[samples]

    for t in range(steps_max):
      num_remaining  = (~stop_flag).sum().item()
      print('number of remaining samples : ',num_remaining )
      if num_remaining  <= 0:
          break

      remaining_label = y[~stop_flag]
      pertbx = []

      for norm in attack_list:
          if norm in norm_params:
              params = norm_params[norm]
              perturbation = pgd(adv_x[~stop_flag], remaining_label, model, norm=norm, is_sample=is_sample, **params)
              print("the number of added features : ", perturbation.sum()/batch_size - adv_x[~stop_flag].sum()/batch_size)
              pertbx.append(perturbation)
          else:
              raise ValueError("Expected 'l1' or 'l2' or 'linf' norm.")


      # here pertbx.shape = a list of (number of attacks  ,(num_remaining ,features))
      pertbx = torch.vstack(pertbx)
      # here pertbx.shape = a tensor (num_remaining *number of attacks samples, features)

      with torch.no_grad():
        remaining_label_ext = torch.cat([remaining_label] * len(attack_list)) #(labels*number of attacks )
        loss, done = get_loss(pertbx, remaining_label_ext,model) #(labels*number of attacks )
        loss = loss.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)
        done = done.reshape(len(attack_list), num_remaining ).permute(1, 0) #(num_remaining ,number of attacks)

        success_flag = torch.any(done, dim=-1) #(num_remaining )
        # for a sample, if there is at least one successful attack, we will select the one with maximum loss;
        # while if no attacks evade the victim successful, all perturbed examples are reminded for selection

        done[~torch.any(done, dim=-1)] = 1 #loss.shape=done.shape=(samples,number of attacks)
        loss = (loss * done.to(torch.float)) + torch.min(loss) * (~done).to(torch.float) #(num_remaining ,number of attacks)
        pertbx = pertbx.reshape(len(attack_list), num_remaining , x.shape[1]).permute([1, 0, 2])#(num_remaining ,attacks,features)
        _, indices = loss.max(dim=-1) # ans:(samples), max loss among attacks which worked, and max loss among all attacks for sample , none of them worked
        adv_x[~stop_flag] = pertbx[torch.arange(num_remaining ), indices]
        a_loss = loss[torch.arange(num_remaining ), indices]
        pre_stop_flag = stop_flag.clone()
        stop_flag[~stop_flag] = (torch.abs(pre_loss[~stop_flag] - a_loss) < varepsilon) | success_flag
        pre_loss[~pre_stop_flag] = a_loss

    return adv_x

In [61]:
adv_x = PGD_Max(X,y, model)
print(X.sum()/batch_size)
print(adv_x.sum()/batch_size)

number of remaining samples :  256
PGD linf: Attack effectiveness 100.000%.
the number of added features :  tensor(5515.8125)
PGD l2: Attack effectiveness 93.359%.
the number of added features :  tensor(8.7344)
PGD l1: Attack effectiveness 100.000%.
the number of added features :  tensor(2.8164)
number of remaining samples :  0
tensor(26.6289)
tensor(5542.4414)


In [14]:
def pgd_step(x, y, model, norm, k, step_length):
    """
    Projected Gradient Descent (PGD) adversarial attack for stepwise.
    :param y: Ground truth labels
    :param x: Feature vector
    :param model: Neural network model
    :param k: Number of steps
    :param step_length: Step size for each iteration
    :param norm: Norm used for perturbation ('linf' or 'l2')
    :return: The adversarial version of x (tensor)(not rounded)
    """
    model.eval()

    criterion = nn.CrossEntropyLoss(reduction='none')

    # Initialize starting point
    x_next = x.clone()

    # Multi-step PGD
    for t in range(k):
        # Forward pass
        x_var = x_next.clone().detach().requires_grad_(True)
        y_model = model(x_var)
        loss = criterion(y_model, y.view(-1).long())

        # Compute gradient
        grad_vars = torch.autograd.grad(loss.mean(), x_var)
        grad_data = grad_vars[0].data
        gradients = grad_data * (x < 0.5)

        # Norm
        if norm == 'linf':
            perturbation = torch.sign(gradients)
        elif norm == 'l2':
            l2norm = torch.linalg.norm(gradients, dim=-1, keepdim=True)
            perturbation = torch.minimum(torch.tensor(1., dtype=x.dtype, device=x.device), gradients / l2norm)
            perturbation[torch.isnan(perturbation)] = 0.
            perturbation[torch.isinf(perturbation)] = 1.
        elif norm == 'l1':
            #ignore the gradient of indice which is updated
            gradients = gradients * (x_next < 0.5)
            val, _ = torch.topk(gradients, 1)
            perturbation = torch.sign(gradients >= val.expand_as(gradients))
            # stop perturbing the examples that are successful to evade the victim
            outputs = model(x_next)
            _, predicted = torch.topk(outputs, k=1)
            done = (predicted != y).squeeze()
            if torch.all(done):
                break
            perturbation[done] = 0.
        else:
            raise ValueError("Expect 'l1' or 'l2' or 'linf' norm.")

        # Update x_next
        x_next = torch.clamp(x_next + perturbation * step_length, min=0., max=1.)

    return x_next

In [27]:
step_checks = 1
steps = 100
is_attacker = False
is_score_round = False
use_random = False
round_threshold = 0.5
x = X
label = y
attack_list = ['linf', 'l2', 'l1']
norm_params = {
    'l1': {'step_length': 1.0},
    'l2': {'step_length': 0.05},
    'linf': {'step_length': 0.002}
}

model.eval()
if not is_attacker:
    step_checks = [1, 10, 25, 50]
    step_check = random.choice(step_checks)
mini_steps = [step_check] * (steps // step_check)
mini_steps = mini_steps + [steps % step_check] if steps % step_check != 0 else mini_steps

print('mini steps = ', mini_steps)

n, red_n = x.size()[0], x.size()[1]
adv_x = x.detach().clone()
pert_x_cont = None
prev_done = None
for i, mini_step in enumerate(mini_steps):
    with torch.no_grad():
        if i == 0 :
            adv_x = get_x0(adv_x, initial_rounding_threshold=round_threshold, is_sample=use_random)

        _, done = get_loss(adv_x, label, model)
    if torch.all(done):
        break
    if i == 0:
        adv_x[~done] = x[~done]  # recompute the perturbation under other penalty factors
        prev_done = done
    else:
        adv_x[~done] = pert_x_cont[~done[~prev_done]]
        prev_done = done

    num_sample_red = torch.sum(~done).item()
    pertbx = []
    for norm in attack_list:
        params = norm_params[norm]
        perturbation = pgd_step(adv_x[~done], label[~done], model, norm=norm, k=mini_step, **params)
        print("the number of added features(not rounded) ", norm,": ", perturbation.sum()/len(adv_x[~done]) - adv_x[~done].sum()/len(adv_x[~done]))
        pertbx.append(perturbation)
    with torch.no_grad():
        n_attacks = len(pertbx)
        pertbx = torch.vstack(pertbx)
        label_ext = torch.cat([label[~done]] * n_attacks)
        if (not is_attacker) and (not is_score_round):
            scores, _done = get_loss(pertbx, label_ext,model)
        else:
            scores, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)
        max_v = scores.amax() if scores.amax() > 0 else 0.
        scores[_done] += max_v
        pertbx = pertbx.reshape(n_attacks, num_sample_red, red_n).permute([1, 0, 2])
        scores = scores.reshape(n_attacks, num_sample_red).permute(1, 0)
        _2, s_idx = scores.max(dim=-1)
        pert_x_cont = pertbx[torch.arange(num_sample_red), s_idx]
        adv_x[~done] = pert_x_cont if not is_attacker else round_x(pert_x_cont, round_threshold)


if is_attacker:
    adv_x = round_x(adv_x, round_threshold)
with torch.no_grad():
    _, done = get_loss(adv_x, label, model)
    print(f"step-wise max: attack effectiveness {done.sum().item() / done.size()[0] * 100:.3f}%.")


mini steps =  [50, 50]
the number of added features(not rounded)  linf :  tensor(539.6423)
the number of added features(not rounded)  l2 :  tensor(116.0846)
the number of added features(not rounded)  l1 :  tensor(2.8275)
step-wise max: attack effectiveness 100.000%.


In [38]:
def StepwiseMax(
    x,
    label,
    model,
    attack_list=["linf", "l2", "l1"],
    step_lengths={"l1": 1.0, "l2": 0.05, "linf": 0.002},
    steps=100,
    random_start=False,
    round_threshold=0.5,
    is_attacker=False,
):
  """
    Stepwise max attack (mixture of pgd-l1, pgd-l2, pgd-linf).

    Args:
        x: Input data tensor (shape: [batch_size, feature_dim])
        label: Ground truth labels tensor (shape: [batch_size])
        model: Victim model
        attack_list: List of attack norms (default: ["linf", "l2", "l1"])
        step_lengths: Dictionary mapping norm to its step length (default: {"l1": 1.0, "l2": 0.05, "linf": 0.002})
        steps: Maximum number of iterations (default: 100)
        random_start: Use random starting point (default: False)
        round_threshold: Threshold for rounding real scalars (default: 0.5)
        is_attacker: Play the role of attacker (default: False)

    Returns:
        Adversarial examples tensor (same shape as x)
  """

  model.eval()

  step_check = 1
  if not is_attacker:
      step_checks = [1, 10, 25, 50]
      step_check = random.choice(step_checks)
      print(f"Step check: {step_check}")

  mini_steps = [step_check] * (steps // step_check)
  if steps % step_check != 0:
      mini_steps.append(steps % step_check)

  n, red_n = x.shape
  adv_x = x.detach().clone()
  pert_x_cont = None
  prev_done = None
  for i, mini_step in enumerate(mini_steps):
      with torch.no_grad():
          if i == 0 :
              adv_x = get_x0(adv_x, initial_rounding_threshold=round_threshold, is_sample=random_start)

          _, done = get_loss(adv_x, label, model)
      if torch.all(done):
          break
      if i == 0:
          adv_x[~done] = x[~done]  # recompute the perturbation under other penalty factors
          prev_done = done
      else:
          adv_x[~done] = pert_x_cont[~done[~prev_done]]
          prev_done = done

      num_sample_red = torch.sum(~done).item()
      pertbx = []
      for norm in attack_list:
          step_length = step_lengths.get(norm, step_lengths["l1"])
          perturbation = pgd_step(adv_x[~done], label[~done], model, norm, mini_step, step_length)
          pertbx.append(perturbation)
      with torch.no_grad():
          pertbx = torch.vstack(pertbx)

          n_attacks = len(attack_list)
          label_ext = torch.cat([label[~done]] * n_attacks)

          if (not is_attacker) and (not is_score_round):
              scores, _done = get_loss(pertbx, label_ext,model)
          else:
              scores, _done = get_loss(round_x(pertbx, round_threshold), label_ext, model)
          max_v = scores.amax() if scores.amax() > 0 else 0.
          scores[_done] += max_v

          pertbx = pertbx.reshape(n_attacks, num_sample_red, red_n).permute([1, 0, 2])
          scores = scores.reshape(n_attacks, num_sample_red).permute(1, 0)
          _2, s_idx = scores.max(dim=-1)
          pert_x_cont = pertbx[torch.arange(num_sample_red), s_idx]
          adv_x[~done] = pert_x_cont if not is_attacker else round_x(pert_x_cont, round_threshold)


  if is_attacker:
      adv_x = round_x(adv_x, round_threshold)
  with torch.no_grad():
      _, done = get_loss(adv_x, label, model)
      print(f"step-wise max: attack effectiveness {done.sum().item() / done.size()[0] * 100:.3f}%.")

  return adv_x

In [40]:
adv_x = StepwiseMax(X, y, model)
print(X.sum()/batch_size)
print(adv_x.sum()/batch_size)

Step check: 1
step-wise max: attack effectiveness 100.000%.
tensor(26.6289)
tensor(46.9497)
